# Experiment D

Ideas:


~~3) Estudar as diferentes distâncias para vetores 1-D em `scipy.spatial.distance`.~~ 

4) De 1, implementar e otimizar os que melhor performam comos portfólios.

5) Implementar como entrada uma empresa nova, conter mapeamento de valores.

Não esquecer de criar o notebook da Second_View, com:
1) Verificar `sklearn.inspection.permutation_importance`.

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [ ]:
%load_ext watermark

In [ ]:
import pandas as pd

In [ ]:
# Run this cell before close.
%watermark
%watermark --iversion
%watermark -b -r -g